---
<center><h1>Basic intro into pandas</h1></center> 
---
---

<center><h2>Working with pandas DataFrames: grouping</h2></center>
---

## Table of Contents

- [Work with pandas DataFrames: grouping](#Work-with-pandas-DataFrames:-grouping)
    * [Splitting of a DataFrame into groups](#Splitting-of-a-DataFrame-into-groups)
    * [Selection and filtering](#Selection-and-filtering)
    * [Aggregation and function application](#Aggregation-and-function-application)
    - [*Exercise 1*](#Exercise-1)

In [145]:
import pandas as pd
import numpy as np
import random

## Work with pandas DataFrames: grouping

[[back to top]](#Table-of-Contents)

We will a werather station dataset so that we can see what is going on when we group.

In [146]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
#Import a COVID data layer.  This layer contains the updated stats for each county in the United States
from arcgis.features import FeatureLayer
mylayer = FeatureLayer(("https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NOAA_METAR_current_wind_speed_direction_v1/FeatureServer/0"))
sdf2 = pd.DataFrame.spatial.from_layer(mylayer)
sdf2.head(5)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
0,1,YBWP,2021-02-24 21:21:00,Weipa City,Australia,21.0,77.0,73.4,89.0,0.0,0,0,NaN,10000.0,1004.9,Scattered Clouds at 240 meters AGL,Automated observation with no human augmentati...,None,NaN,-12.68,141.92,"{""x"": 15798462.1334, ""y"": -1423196.3836000003,..."
1,2,KDTA,2021-02-24 21:21:00,Delta,"Utah, United States Of America",1452.0,39.2,19.4,44.0,40.0,28,39,30.1,16000.0,1019.3,Overcast Cloud Deck at 1500 meters AGL,Automated observation with no human augmentati...,None,NaN,39.38,-112.50,"{""x"": -12523442.714200001, ""y"": 4776250.407300..."
2,3,PKWA,2021-02-24 21:20:00,Kwajalein/Buchol,"Marshall Islands, United States Of America",6.0,83.1,74.8,76.0,80.0,30,0,NaN,16000.0,1009.1,Scattered Clouds at 820 meters AGL,Automated observation with no human augmentati...,None,90.4,8.72,167.72,"{""x"": 18670504.995800003, ""y"": 974475.15549999..."
3,4,PAII,2021-02-24 21:20:00,Egegik,"Alaska, United States Of America",31.0,33.8,32.0,93.0,240.0,22,30,24.3,13000.0,1005.4,Scattered Clouds at 90 meters AGL,Light Rain,Hourly Precipitation Amount: trace,NaN,58.18,-157.38,"{""x"": -17519461.461, ""y"": 8005225.349699996, ""..."
4,5,PAHL,2021-02-24 21:20:00,Huslia,"Alaska, United States Of America",49.0,-2.2,-7.6,77.0,70.0,24,37,-22.2,4800.0,1004.0,"Few Clouds at 550 meters AGL, Broken Clouds at...",Light Snow,Hourly Precipitation Amount: trace,NaN,65.70,-156.35,"{""x"": -17404802.3855, ""y"": 9795217.7636, ""spat..."


In [147]:
sdfUSA = sdf2.loc[sdf2["COUNTRY"].apply(lambda x: "United States Of America" in str(x))]
sdfUSA["State"] = sdfUSA['COUNTRY'].apply(lambda x: str(x).split(",")[0])
sdfUSA["COUNTRY"] = sdfUSA['COUNTRY'].apply(lambda x: str(x).split(",")[1])
sdfUSA

D66140B0-0277-45FF-AE18-A787BE3D4AAF:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D66140B0-0277-45FF-AE18-A787BE3D4AAF:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
1,2,KDTA,2021-02-24 21:21:00,Delta,United States Of America,1452.0,39.2,19.4,44.0,40.0,28,39,30.1,16000.0,1019.3,Overcast Cloud Deck at 1500 meters AGL,Automated observation with no human augmentati...,None,NaN,39.38,-112.50,"{""x"": -12523442.714200001, ""y"": 4776250.407300...",Utah
2,3,PKWA,2021-02-24 21:20:00,Kwajalein/Buchol,United States Of America,6.0,83.1,74.8,76.0,80.0,30,0,NaN,16000.0,1009.1,Scattered Clouds at 820 meters AGL,Automated observation with no human augmentati...,None,90.4,8.72,167.72,"{""x"": 18670504.995800003, ""y"": 974475.15549999...",Marshall Islands
3,4,PAII,2021-02-24 21:20:00,Egegik,United States Of America,31.0,33.8,32.0,93.0,240.0,22,30,24.3,13000.0,1005.4,Scattered Clouds at 90 meters AGL,Light Rain,Hourly Precipitation Amount: trace,NaN,58.18,-157.38,"{""x"": -17519461.461, ""y"": 8005225.349699996, ""...",Alaska
4,5,PAHL,2021-02-24 21:20:00,Huslia,United States Of America,49.0,-2.2,-7.6,77.0,70.0,24,37,-22.2,4800.0,1004.0,"Few Clouds at 550 meters AGL, Broken Clouds at...",Light Snow,Hourly Precipitation Amount: trace,NaN,65.70,-156.35,"{""x"": -17404802.3855, ""y"": 9795217.7636, ""spat...",Alaska
7,8,KMOR,2021-02-24 21:20:00,Morristown/Moore,United States Of America,393.0,68.0,28.4,23.0,270.0,28,41,NaN,14000.0,1014.5,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,36.18,-83.37,"{""x"": -9280705.9474, ""y"": 4325417.444600001, ""...",Tennessee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4728,4729,KMVE,2021-02-24 20:35:00,Montevideo,United States Of America,319.0,33.8,21.2,59.0,350.0,9,0,28.5,16000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,44.97,-95.72,"{""x"": -10655501.6587, ""y"": 5616799.836000003, ...",Minnesota
4729,4730,K54J,2021-02-24 20:35:00,De Funiak Sprngs,United States Of America,87.0,70.7,46.8,42.0,0.0,0,0,NaN,16000.0,1019.6,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,75.2,30.73,-86.15,"{""x"": -9590174.1318, ""y"": 3597733.9235000014, ...",Florida
4739,4740,KHSA,2021-02-24 20:30:00,Bay St Louis,United States Of America,6.0,69.8,55.4,60.0,160.0,15,0,NaN,16000.0,1018.9,Overcast Cloud Deck at 1800 meters AGL,No significant weather present at this time.,None,NaN,30.37,-89.45,"{""x"": -9957528.4515, ""y"": 3551199.1072999984, ...",Mississippi
4742,4743,PAMB,2021-02-24 20:26:00,Manokotak,United States Of America,28.0,32.0,30.2,93.0,240.0,13,0,24.8,6400.0,1002.4,"Broken Clouds at 90 meters AGL, Overcast Cloud...",Mist,* Maintenance REQUIRED - readings may be compr...,NaN,58.98,-159.05,"{""x"": -17705365.0107, ""y"": 8176065.373599999, ...",Alaska


In [148]:
sdfUSA.dtypes

OBJECTID                 int64
ICAO                    object
OBS_DATETIME    datetime64[ns]
STATION_NAME            object
COUNTRY                 object
ELEVATION              float64
TEMP                   float64
DEW_POINT              float64
R_HUMIDITY             float64
WIND_DIRECT            float64
WIND_SPEED               int64
WIND_GUST                int64
WIND_CHILL             float64
VISIBILITY             float64
PRESSURE               float64
SKY_CONDTN              object
WEATHER                 object
REMARKS                 object
HEAT_INDEX             float64
LATITUDE               float64
LONGITUDE              float64
SHAPE                 geometry
State                   object
dtype: object

So, let’s shortly describe what you will know after reading the blog post:
1.	how to split a DataFrame into a group;
2.	how select a group in the grouped DataFrame and filter the grouping data;
3.	how aggregate grouping data and how apply many function at once to the grouping data;


### Splitting of a DataFrame into groups

[[back to top]](#Table-of-Contents)

pandas DataFrames can be split on any of their axes. Grouping denotes the providence of a mapping of labels to group names. Method `groupby()` is provided in pandas for grouping. The `groupby()` function returns a `GroupBy` object, but essentially describes how the rows of the original data set has been split. 


In [231]:
sdfUSA_grouped = sdfUSA.groupby('State')
sdfUSA_grouped

In [232]:
sdfUSA_grouped['COUNTRY']

Use `list()` to show what a grouping looks like

In [233]:
list(sdfUSA_grouped)

[('Alabama',       OBJECTID  ...    State
1132      1133  ...  Alabama
1147      1148  ...  Alabama
1150      1151  ...  Alabama
1154      1155  ...  Alabama
1161      1162  ...  Alabama
1180      1181  ...  Alabama
1345      1346  ...  Alabama
1372      1373  ...  Alabama
1381      1382  ...  Alabama
1450      1451  ...  Alabama
1470      1471  ...  Alabama
1532      1533  ...  Alabama
1535      1536  ...  Alabama
1607      1608  ...  Alabama
1721      1722  ...  Alabama
1728      1729  ...  Alabama
1818      1819  ...  Alabama
1929      1930  ...  Alabama
2015      2016  ...  Alabama
2019      2020  ...  Alabama
2222      2223  ...  Alabama
2297      2298  ...  Alabama
2314      2315  ...  Alabama
2427      2428  ...  Alabama
2442      2443  ...  Alabama
2471      2472  ...  Alabama
2514      2515  ...  Alabama
2515      2516  ...  Alabama
2563      2564  ...  Alabama
2814      2815  ...  Alabama
2822      2823  ...  Alabama
2944      2945  ...  Alabama
2966      2967  ...  Alabama
3

As it can be easily seen, the `GroupBy` object is a dictionary whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group.

In [234]:
for key, value in list(sdfUSA_grouped)[:3]:
# if you want to see more values uncomment row below and comment the previous row
#for key, value in short_grouped:
    print (key)
    print (value)

Alabama
      OBJECTID  ...    State
1132      1133  ...  Alabama
1147      1148  ...  Alabama
1150      1151  ...  Alabama
1154      1155  ...  Alabama
1161      1162  ...  Alabama
1180      1181  ...  Alabama
1345      1346  ...  Alabama
1372      1373  ...  Alabama
1381      1382  ...  Alabama
1450      1451  ...  Alabama
1470      1471  ...  Alabama
1532      1533  ...  Alabama
1535      1536  ...  Alabama
1607      1608  ...  Alabama
1721      1722  ...  Alabama
1728      1729  ...  Alabama
1818      1819  ...  Alabama
1929      1930  ...  Alabama
2015      2016  ...  Alabama
2019      2020  ...  Alabama
2222      2223  ...  Alabama
2297      2298  ...  Alabama
2314      2315  ...  Alabama
2427      2428  ...  Alabama
2442      2443  ...  Alabama
2471      2472  ...  Alabama
2514      2515  ...  Alabama
2515      2516  ...  Alabama
2563      2564  ...  Alabama
2814      2815  ...  Alabama
2822      2823  ...  Alabama
2944      2945  ...  Alabama
2966      2967  ...  Alabama
3037  

Besides, you may to see the first and the last item of each group

In [235]:
sdfUSA_grouped.first().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,
Alabama,1133,KSXS,2021-02-24 15:58:00,Ft Rucker/Shell,United States Of America,120.0,64.6,43.5,46.0,240.0,9,0,NaN,14000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,78.8,31.37,-85.85,"{""x"": -9556778.2846, ""y"": 3680894.4886000007, ..."
Alaska,2,PAJZ,2021-02-24 16:05:00,Koliganek,United States Of America,80.0,26.6,26.6,100.0,120.0,24,0,14.7,4000.0,1003.0,"Few Clouds at 490 meters AGL, Overcast Cloud D...","Light Snow, Mist",* Maintenance REQUIRED - readings may be compr...,NaN,59.73,-157.27,"{""x"": -17507216.3171, ""y"": 8339869.139899999, ..."
Arizona,1110,KNOZ,2021-02-24 15:59:00,Wellton,United States Of America,78.0,60.1,27.0,28.0,150.0,15,0,30.3,16000.0,1016.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,32.53,-114.50,"{""x"": -12746081.695799999, ""y"": 3833084.206799..."
Arkansas,1140,KROG,2021-02-24 15:58:00,Rogers,United States Of America,407.0,53.1,35.1,50.0,10.0,15,0,NaN,16000.0,1015.6,Overcast Cloud Deck at 1000 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,36.37,-94.10,"{""x"": -10475164.0836, ""y"": 4351652.965599999, ..."
California,14,KNSI,2021-02-24 16:04:00,San Nicolas Isla,United States Of America,161.0,54.0,53.1,97.0,350.0,11,0,66.8,1200.0,1016.9,"Scattered Clouds at 90 meters AGL, Broken Clou...",Mist,* Maintenance REQUIRED - readings may be compr...,NaN,33.23,-119.45,"{""x"": -13297113.1753, ""y"": 3925872.5605000034,..."
Colorado,1112,KMYP,2021-02-24 15:59:00,Monarch Pass,United States Of America,3629.0,10.4,6.8,85.0,240.0,31,0,-8.2,16000.0,1020.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,38.48,-106.32,"{""x"": -11835488.2611, ""y"": 4647457.398199998, ..."
Connecticut,1445,KGON,2021-02-24 15:56:00,Groton/New Londo,United States Of America,2.0,46.0,35.1,65.0,210.0,19,0,40.6,16000.0,1016.9,Sky Clear below 3700 meters AGL,No significant weather present at this time.,Wind Shift At 15:30Z,NaN,41.33,-72.05,"{""x"": -8020569.3116999995, ""y"": 5061139.118699..."
Delaware,1485,KDOV,2021-02-24 15:56:00,Dover Afb,United States Of America,6.0,53.8,32.9,45.0,180.0,15,0,45.4,16000.0,1019.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,39.12,-75.47,"{""x"": -8401281.9702, ""y"": 4738875.1483, ""spati..."
Florida,15,KMKY,2021-02-24 16:04:00,Marco Island,United States Of America,2.0,78.8,69.8,74.0,170.0,15,0,NaN,16000.0,1022.0,Scattered Clouds at 580 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,82.5,26.00,-81.67,"{""x"": -9091462.8131, ""y"": 2999080.943500001, ""..."


In [236]:
sdfUSA_grouped.last().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,
Alabama,4995,KPRN,2021-02-24 14:58:00,Greenville,United States Of America,137.0,57.0,35.1,44.0,0.0,6,0,NaN,16000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 1.4 mb,78.8,31.85,-86.62,"{""x"": -9642494.2925, ""y"": 3743636.8615999967, ..."
Alaska,5000,PANW,2021-02-24 14:56:00,New Stuyahok,United States Of America,100.0,26.1,26.1,100.0,100.0,13,0,17.5,6400.0,1002.0,"Few Clouds at 340 meters AGL, Scattered Clouds...","Light Snow, Mist",* Maintenance REQUIRED - readings may be compr...,NaN,59.45,-157.32,"{""x"": -17512782.2916, ""y"": 8278291.432099998, ..."
Arizona,3958,KRYN,2021-02-24 15:45:00,Tucson/Ryan Fld,United States Of America,731.0,51.8,12.2,20.0,100.0,13,0,40.2,16000.0,1019.3,Sky Clear below 3700 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,32.15,-111.17,"{""x"": -12375387.7915, ""y"": 3783016.603500001, ..."
Arkansas,4146,KMWT,2021-02-24 15:14:00,Mount Ida,United States Of America,220.0,57.0,50.0,77.0,0.0,7,0,NaN,16000.0,1014.2,"Broken Clouds at 400 meters AGL, Broken Clouds...",Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,34.55,-93.58,"{""x"": -10417277.9484, ""y"": 4102894.8566000015,..."
California,5043,KNFG,2021-02-24 14:55:00,Camp Pendleton,United States Of America,21.0,39.9,28.9,65.0,0.0,0,0,60.5,16000.0,1017.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,33.30,-117.35,"{""x"": -13063342.2446, ""y"": 3935191.9821000025,..."
Colorado,4994,KFCS,2021-02-24 14:59:00,Fort Carson,United States Of America,1789.0,28.4,17.6,63.0,170.0,26,0,16.5,16000.0,1020.3,Broken Clouds at 910 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,38.67,-104.77,"{""x"": -11662943.0504, ""y"": 4674511.5361, ""spat..."
Connecticut,3402,KBDL,2021-02-24 15:51:00,Windsor Locks,United States Of America,52.0,44.1,27.0,51.0,300.0,15,0,39.0,16000.0,1015.9,"Few Clouds at 1200 meters AGL, Few Clouds at 3...",No significant weather present at this time.,Wind Shift At 15:30Z,NaN,41.93,-72.68,"{""x"": -8090700.5909, ""y"": 5150499.544100001, ""..."
Delaware,3382,KILG,2021-02-24 15:51:00,Wilmington,United States Of America,22.0,48.9,33.1,54.0,140.0,13,0,45.4,16000.0,1018.9,Sky Clear below 3700 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,39.67,-75.60,"{""x"": -8415753.504, ""y"": 4818102.8226, ""spatia..."
Florida,5047,KNIP,2021-02-24 14:53:00,Jacksonville Nas,United States Of America,3.0,62.1,51.1,67.0,0.0,0,0,NaN,16000.0,1023.4,Scattered Clouds at 1000 meters AGL,No significant weather present at this time.,3-Hour Pressure Increasing: 1.5 mb,77.0,30.23,-81.67,"{""x"": -9091462.8131, ""y"": 3533148.591300003, ""..."


### Selection and filtering

[[back to top]](#Table-of-Contents)

Functions of descriptive statistic like `sum()`, `count()`, `max()`, `min()`, `mean()` can be quickly applied to the `GroupBy` object to obtain summary statistics for each group – an immensely useful function. The same statement is valid also for functions like `describe()` which return general information about an object. 

Let’s show a few examples of descriptive statistic function application to the `GroupBy` object

In [237]:
sdfUSA_grouped.count().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,
Alabama,41,41,41,41,41,41,41,41,41,41,41,41,0,40,41,40,41,5,1,41,41,41
Alaska,128,128,128,128,128,128,125,120,120,123,128,128,123,125,127,124,128,73,0,128,128,128
Arizona,42,42,42,42,42,42,42,42,42,42,42,42,14,41,42,42,42,5,0,42,42,42
Arkansas,36,36,36,36,36,36,36,36,36,35,36,36,0,35,36,36,36,3,0,36,36,36
California,131,131,131,131,131,131,130,130,130,131,131,131,78,130,131,130,131,18,0,131,131,131
Colorado,64,64,64,64,64,64,64,64,64,64,64,64,64,63,63,63,64,4,0,64,64,64
Connecticut,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1,0,10,10,10
Delaware,4,4,4,4,4,4,4,4,4,4,4,4,1,4,4,4,4,3,0,4,4,4
Florida,95,95,95,95,95,93,94,94,94,94,95,95,0,94,95,94,95,22,52,95,95,95


In [238]:
sdfUSA_grouped.sum().add_prefix('sum_of_').head(10)
# calculate the sum only for numeric columns, 
# all columns with other types will be ignored and missed
# we have add also common prefics to all columns. It's very convenient

,sum_of_OBJECTID,sum_of_ELEVATION,sum_of_TEMP,sum_of_DEW_POINT,sum_of_R_HUMIDITY,sum_of_WIND_DIRECT,sum_of_WIND_SPEED,sum_of_WIND_GUST,sum_of_WIND_CHILL,sum_of_VISIBILITY,sum_of_PRESSURE,sum_of_HEAT_INDEX,sum_of_LATITUDE,sum_of_LONGITUDE
State,,,,,,,,,,,,,,
Alabama,90258,5525.0,2572.8,1513.7,1614.0,5780.0,367,0,0.0,634000.0,41874.9,78.8,1337.00,-3549.71
Alaska,231927,9685.0,1597.3,1254.4,10028.0,15270.0,2031,755,642.1,1323300.0,127976.0,0.0,7947.34,-19448.38
Arizona,100031,39608.0,2132.7,716.2,1178.0,4190.0,316,65,649.4,649000.0,42791.4,0.0,1417.80,-4695.26
Arkansas,88343,6070.0,2098.6,1573.4,2131.0,6140.0,547,199,0.0,556000.0,36545.2,0.0,1268.15,-3332.84
California,333401,39875.0,6184.2,4037.0,7379.0,14320.0,772,218,3907.9,2208100.0,133672.3,0.0,4729.19,-15665.82
Colorado,150536,132024.0,1656.1,754.6,3615.0,9180.0,839,314,1575.0,1008000.0,64253.1,0.0,2505.33,-6758.69
Connecticut,28734,632.0,455.0,292.6,533.0,1710.0,127,50,414.8,160000.0,10169.2,0.0,414.92,-727.51
Delaware,10097,60.0,210.3,132.0,190.0,700.0,56,0,45.4,64000.0,4077.8,0.0,156.69,-302.04
Florida,236043,1465.0,6726.3,5298.6,5646.0,9920.0,899,50,0.0,1477600.0,97148.2,4080.8,2703.97,-7832.29


In [150]:
sdfUSA_grouped.mean()

,OBJECTID,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,HEAT_INDEX,LATITUDE,LONGITUDE
State,,,,,,,,,,,,,,
Alabama,2605.463415,134.756098,72.778049,34.195122,25.926829,195.853659,16.560976,11.024390,NaN,15756.097561,1017.870732,73.272222,32.609756,-86.578293
Alaska,2720.780303,83.174242,14.385271,11.612097,82.274194,108.346457,17.560606,8.833333,6.672441,10378.906250,1005.492424,NaN,62.099242,-151.829318
Arizona,2892.780488,958.268293,67.956098,13.585366,13.243902,174.000000,14.951220,13.219512,42.700000,16170.731707,1016.795122,70.883333,33.762683,-111.779512
Arkansas,3348.371429,171.228571,67.051429,47.674286,51.314286,209.705882,14.228571,8.628571,NaN,15823.529412,1014.562857,75.750000,35.205143,-92.654000
California,3250.122137,299.297710,64.882308,29.164615,30.107692,215.801527,16.396947,9.679389,43.250000,15950.769231,1019.913740,72.682353,36.052595,-119.565267
Colorado,2422.953125,2067.593750,37.045313,9.439063,33.046875,150.645161,15.390625,15.703125,32.068852,16000.000000,1017.403175,NaN,39.178594,-105.612969
Connecticut,3801.600000,63.200000,49.090000,28.920000,45.700000,205.555556,18.800000,17.600000,43.737500,16000.000000,1013.100000,NaN,41.492000,-72.751000
Delaware,3249.500000,15.000000,58.700000,33.275000,38.500000,192.500000,20.750000,8.750000,NaN,14750.000000,1015.125000,NaN,39.172500,-75.510000
Florida,3109.242105,15.752688,76.001064,55.365957,51.319149,163.191489,14.263158,2.810526,NaN,15754.736842,1019.736842,79.032432,28.462842,-82.445158


In [151]:
sdfUSA.groupby(['State'])['TEMP'].mean()
# calculate sum for a single column
# it works more quickly then movies.groupby(['State']).mean()['TEMP'] becuase it sheads the data it does not need before the mean operation

State
Alabama               73.247619
Alaska                15.070492
Arizona               69.173171
Arkansas              67.400000
California            65.613740
Colorado              37.889231
Connecticut           48.080000
Delaware              58.000000
Florida               75.467391
Georgia               72.738272
Guam                  77.750000
Hawaii                76.392857
Idaho                 33.258333
Illinois              47.674138
Indiana               46.114634
Iowa                  36.960000
Johnston/Wake/Xmas    81.300000
Kansas                49.120370
Kentucky              64.663158
Louisiana             72.792157
Maine                 40.752941
Marshall Islands      83.450000
Maryland              58.272727
Massachusetts         47.390909
Michigan              36.717333
Minnesota             32.533673
Mississippi           72.145455
Missouri              53.055814
Montana               31.405263
Nebraska              40.628205
Nevada                47.968182
Ne

In [152]:
sdfUSA.groupby(['State', 'STATION_NAME']).sum()
# it is possible to group by many columns
# pay your attention that the changing of the order of 
# columns for grouping in 
# its list also changes the result table

OBJECTID  ELEVATION  ...  LATITUDE  LONGITUDE
State   STATION_NAME                           ...                     
Alabama Alabaster             3954      172.0  ...     33.18     -86.78
        Albertville Muni       966      313.0  ...     34.23     -86.25
        Alexander/Russel       938      198.0  ...     32.92     -85.97
        Andalusia/Opp Ar      3327       93.0  ...     31.32     -86.40
        Anniston              4069      187.0  ...     33.58     -85.85
...                            ...        ...  ...       ...        ...
Wyoming Torrington            3635     1285.0  ...     42.07    -104.15
        Warren Air Base       2984     1877.0  ...     41.13    -104.87
        Wheatland              734     1452.0  ...     42.05    -104.93
        Worland               3607     1281.0  ...     43.97    -107.95
        Yellowstone Lake      3137     2377.0  ...     44.55    -110.42

[2358 rows x 14 columns]

It is possible also to show all groups

In [153]:
sdfUSA_grouped.groups

{'Alabama': [374, 484, 490, 584, 691, 775, 778, 978, 1050, 1065, 1177, 1192, 1220, 1260, 1261, 1310, 3199, 3207, 3212, 3215, 3219, 3225, 3244, 3373, 3403, 3412, 3481, 3500, 3561, 3564, 3873, 3881, 4006, 4028, 4100, 4123, 4191, 4211, 4279, 4282, 4305], 'Alaska': [1, 2, 10, 292, 293, 299, 300, 301, 302, 313, 314, 315, 316, 322, 323, 324, 332, 333, 335, 336, 1345, 1357, 1364, 1365, 1366, 1402, 1408, 1409, 1418, 1419, 1420, 1427, 1428, 1432, 1433, 1434, 1438, 1439, 1440, 1441, 1443, 1446, 3193, 3194, 3245, 3255, 3259, 3260, 3261, 3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 3271, 3272, 3273, 3274, 3275, 3276, 3277, 3278, 3279, 3280, 3281, 3282, 3283, 3284, 3285, 3286, 3287, 3288, 3289, 3290, 3291, 3292, 3293, 3294, 3295, 3296, 3297, 3298, 3299, 3581, 3582, 3583, 3584, 3585, 3586, 3692, 3693, 3694, 3808, 3809, 3810, 3811, ...], 'Arizona': [320, 405, 460, 482, 602, 603, 767, 845, 1064, 1114, 1162, 1170, 1213, 3185, 3210, 3213, 3220, 3227, 3246, 3249, 3449, 3529, 3553, 3707, 3728, 37

and display the content only of necessary group

In [154]:
sdfUSA_grouped.get_group("Mississippi")

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
378,379,KVKS,2021-02-24 20:15:00,Vicksburg,United States Of America,31.0,75.2,57.2,54.0,200.0,31,48,NaN,16000.0,1015.6,Overcast Cloud Deck at 1500 meters AGL,Automated observation with no human augmentati...,None,77.6,32.23,-90.93,"{""x"": -10122281.2978, ""y"": 3793539.7114000022,...",Mississippi
389,390,KUTA,2021-02-24 20:15:00,Tunica,United States Of America,58.0,60.8,50.0,68.0,190.0,19,0,NaN,16000.0,1015.2,Scattered Clouds at 1500 meters AGL,Automated observation with no human augmentati...,None,NaN,34.68,-90.35,"{""x"": -10057715.9932, ""y"": 4120479.0078999996,...",Mississippi
391,392,KUOX,2021-02-24 20:15:00,Oxford,United States Of America,123.0,64.4,46.4,52.0,240.0,15,24,NaN,14000.0,1016.2,Overcast Cloud Deck at 1000 meters AGL,Automated observation with no human augmentati...,None,NaN,34.38,-89.53,"{""x"": -9966434.0107, ""y"": 4079941.589400001, ""...",Mississippi
524,525,KRNV,2021-02-24 20:15:00,Cleveland,United States Of America,42.0,66.0,55.8,70.0,220.0,28,35,NaN,16000.0,1015.9,Overcast Cloud Deck at 1300 meters AGL,Automated observation with no human augmentati...,None,NaN,33.77,-90.75,"{""x"": -10102243.7895, ""y"": 3997960.298299998, ...",Mississippi
581,582,KPMU,2021-02-24 20:15:00,Batesville,United States Of America,67.0,64.4,46.4,52.0,210.0,15,26,NaN,16000.0,1015.6,"Scattered Clouds at 850 meters AGL, Overcast C...",Automated observation with no human augmentati...,None,NaN,34.37,-89.90,"{""x"": -10007622.2223, ""y"": 4078592.8506999984,...",Mississippi
704,705,KM40,2021-02-24 20:15:00,Aberdeen/Amory,United States Of America,68.0,77.0,30.2,18.0,210.0,20,0,NaN,16000.0,1016.2,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,74.4,33.87,-88.48,"{""x"": -9849548.5454, ""y"": 4011359.5310999975, ...",Mississippi
717,718,KLUL,2021-02-24 20:15:00,Laurel,United States Of America,74.0,76.3,47.1,36.0,220.0,17,30,NaN,16000.0,1017.6,Scattered Clouds at 1500 meters AGL,Automated observation with no human augmentati...,None,76.8,31.67,-89.17,"{""x"": -9926358.994, ""y"": 3720070.5239999965, ""...",Mississippi
766,767,KJVW,2021-02-24 20:15:00,Raymond,United States Of America,73.0,75.2,55.4,50.0,200.0,17,0,NaN,16000.0,1016.2,Scattered Clouds at 1500 meters AGL,Automated observation with no human augmentati...,None,77.4,32.30,-90.42,"{""x"": -10065508.3575, ""y"": 3802755.0319000036,...",Mississippi
1048,1049,KCRX,2021-02-24 20:15:00,Corinth/R Turner,United States Of America,128.0,73.4,33.8,23.0,220.0,26,39,NaN,16000.0,1014.5,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,73.3,34.92,-88.60,"{""x"": -9862906.8843, ""y"": 4153014.7710999995, ...",Mississippi
1072,1073,KCKM,2021-02-24 20:15:00,Clarksdale,United States Of America,52.0,64.4,53.6,68.0,200.0,15,0,NaN,16000.0,1015.2,None,Automated observation with no human augmentati...,None,NaN,34.30,-90.52,"{""x"": -10076640.3066, ""y"": 4069156.1828000024,...",Mississippi


In [155]:
sdfUSA_grouped.get_group("Mississippi").mean()

D66140B0-0277-45FF-AE18-A787BE3D4AAF:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.


OBJECTID        2557.636364
ELEVATION         73.212121
TEMP              72.145455
DEW_POINT         48.466667
R_HUMIDITY        45.787879
WIND_DIRECT      210.000000
WIND_SPEED        19.242424
WIND_GUST         17.545455
WIND_CHILL              NaN
VISIBILITY     15648.484848
PRESSURE        1016.733333
HEAT_INDEX        76.156522
LATITUDE          32.644242
LONGITUDE        -89.681515
dtype: float64

We have just one helpful and powerful option for you! It is `filter()`.

Suppose you need remain only those groups where there are less than 5 items. Sure, you may hardly calculate amount of items in each group and then remove not satisfying for condition. But it can be done very easily.

In [156]:
sdfUSA_grouped.filter(lambda x: len(x) < 5).sort_values("State")

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
1282,1283,K33N,2021-02-24 20:15:00,Cheswold/Dover,United States Of America,17.0,59.0,30.2,33.0,210.0,20,35,NaN,16000.0,1014.5,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,39.22,-75.60,"{""x"": -8415753.504, ""y"": 4753233.857000001, ""s...",Delaware
3514,3515,KDOV,2021-02-24 19:56:00,Dover Afb,United States Of America,6.0,60.8,33.8,36.0,190.0,19,0,NaN,11000.0,1015.9,Sky Clear below 3700 meters AGL,No significant weather present at this time.,None,NaN,39.12,-75.47,"{""x"": -8401281.9702, ""y"": 4738875.1483, ""spati...",Delaware
3759,3760,KGED,2021-02-24 19:54:00,Georgetown,United States Of America,15.0,61.0,35.1,38.0,200.0,22,0,NaN,16000.0,1015.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,38.68,-75.37,"{""x"": -8390150.0211, ""y"": 4675937.423600003, ""...",Delaware
4439,4440,KILG,2021-02-24 19:51:00,Wilmington,United States Of America,22.0,54.0,34.0,47.0,170.0,22,0,NaN,16000.0,1014.2,Sky Clear below 3700 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,39.67,-75.60,"{""x"": -8415753.504, ""y"": 4818102.8226, ""spatia...",Delaware
312,313,PGUM,2021-02-24 20:17:00,Agana,United States Of America,74.0,79.0,71.1,77.0,80.0,13,0,NaN,16000.0,1010.5,"Scattered Clouds at 640 meters AGL, Broken Clo...",No significant weather present at this time.,None,83.2,13.48,144.80,"{""x"": 16119062.266900003, ""y"": 1514624.8880999...",Guam
3580,3581,PGUA,2021-02-24 19:55:00,Mariana/Andersen,United States Of America,159.0,76.6,70.0,80.0,100.0,20,0,NaN,16000.0,1010.1,Scattered Clouds at 520 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,80.3,13.57,144.92,"{""x"": 16132420.605800003, ""y"": 1524929.4087000...",Guam
2033,2034,PLCH,2021-02-24 20:00:00,Christmas/Cassid,United States Of America,5.0,80.6,75.2,84.0,110.0,22,0,NaN,10000.0,1008.9,Scattered Clouds at 580 meters AGL,No significant weather present at this time.,None,86.2,1.98,-157.35,"{""x"": -17516121.8763, ""y"": 220456.47509999946,...",Johnston/Wake/Xmas
3191,3192,PWAK,2021-02-24 19:58:00,Wake Island Airf,United States Of America,7.0,77.9,69.1,74.0,90.0,6,0,NaN,10000.0,1012.5,Few Clouds at 790 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,81.3,19.28,166.62,"{""x"": 18548053.556, ""y"": 2187929.2760000005, ""...",Johnston/Wake/Xmas
17,18,PKWA,2021-02-24 20:20:00,Kwajalein/Buchol,United States Of America,6.0,81.1,75.4,83.0,80.0,33,0,NaN,16000.0,1008.8,"Scattered Clouds at 700 meters AGL, Scattered ...",Automated observation with no human augmentati...,None,87.4,8.72,167.72,"{""x"": 18670504.995800003, ""y"": 974475.15549999...",Marshall Islands
4402,4403,PKMR,2021-02-24 19:51:00,Majuro Atoll/Nws,United States Of America,7.0,81.7,74.7,79.0,70.0,13,0,NaN,24000.0,1007.8,"Few Clouds at 460 meters AGL, Scattered Clouds...",No significant weather present at this time.,None,87.8,7.08,171.38,"{""x"": 19077934.3322, ""y"": 790155.4248000011, ""...",Marshall Islands


One more example:

In [157]:
sdfUSA_grouped.filter(lambda x: x['TEMP'].mean() > 50 and x['R_HUMIDITY'].mean() > 50)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
17,18,PKWA,2021-02-24 20:20:00,Kwajalein/Buchol,United States Of America,6.0,81.1,75.4,83.0,80.0,33,0,NaN,16000.0,1008.8,"Scattered Clouds at 700 meters AGL, Scattered ...",Automated observation with no human augmentati...,None,87.4,8.72,167.72,"{""x"": 18670504.995800003, ""y"": 974475.15549999...",Marshall Islands
72,73,KATP,2021-02-24 20:20:00,Atlantis Oilp,United States Of America,50.0,77.0,68.0,74.0,0.0,0,0,NaN,16000.0,1018.6,"Few Clouds at 460 meters AGL, Scattered Clouds...",Automated observation with no human augmentati...,None,80.2,27.20,-90.03,"{""x"": -10022093.7561, ""y"": 3148481.4043999985,...",Louisiana
312,313,PGUM,2021-02-24 20:17:00,Agana,United States Of America,74.0,79.0,71.1,77.0,80.0,13,0,NaN,16000.0,1010.5,"Scattered Clouds at 640 meters AGL, Broken Clo...",No significant weather present at this time.,None,83.2,13.48,144.80,"{""x"": 16119062.266900003, ""y"": 1514624.8880999...",Guam
341,342,KZPH,2021-02-24 20:15:00,Zephyrhills,United States Of America,26.0,80.6,59.7,49.0,230.0,11,0,NaN,16000.0,1020.0,"Scattered Clouds at 1300 meters AGL, Scattered...",Automated observation with no human augmentati...,None,81.2,28.23,-82.15,"{""x"": -9144896.1687, ""y"": 3278002.5964, ""spati...",Florida
352,353,KX60,2021-02-24 20:15:00,Williston,United States Of America,21.0,77.9,44.6,31.0,260.0,22,0,NaN,16000.0,1020.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,77.2,29.35,-82.47,"{""x"": -9180518.4057, ""y"": 3420269.0613999963, ...",Florida
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4665,4666,KBCT,2021-02-24 19:47:00,Boca Raton,United States Of America,3.0,78.8,68.0,70.0,120.0,19,0,NaN,16000.0,1019.3,Sky Clear below 3700 meters AGL,No significant weather present at this time.,None,82.0,26.38,-80.10,"{""x"": -8916691.2125, ""y"": 3046222.2029, ""spati...",Florida
4671,4672,PHJH,2021-02-24 19:45:00,Lahaina/West Mau,United States Of America,67.0,77.0,59.0,54.0,120.0,20,39,NaN,24000.0,1016.2,"Few Clouds at 1100 meters AGL, Few Clouds at 1...",No significant weather present at this time.,None,78.8,20.97,-156.67,"{""x"": -17440424.6226, ""y"": 2388301.7677999996,...",Hawaii
4686,4687,KSCF,2021-02-24 19:45:00,S Marsh268 Oilp,United States Of America,22.0,68.0,60.8,78.0,0.0,9,22,NaN,16000.0,1018.6,Overcast Cloud Deck at 1200 meters AGL,Automated observation with no human augmentati...,None,NaN,29.12,-91.87,"{""x"": -10226921.6192, ""y"": 3390928.229800001, ...",Louisiana
4695,4696,KCGF,2021-02-24 19:45:00,Clevlnd/Cuyahoga,United States Of America,265.0,51.8,33.8,50.0,240.0,31,54,NaN,32000.0,1005.1,"Broken Clouds at 2400 meters AGL, Broken Cloud...",No significant weather present at this time.,None,NaN,41.57,-81.47,"{""x"": -9069198.9149, ""y"": 5096783.584700003, ""...",Ohio


### Aggregation

[[back to top]](#Table-of-Contents)

We have seen previously, pandas allows to apply any function to a Series or to a DataFrame. Grouping is not an exception and you may use the same method `apply()` directly with `GroupBy` objects.  Aggregation can be done via the `aggregate()` or equivalently `agg()` method

In [158]:
sdfUSA_grouped.agg(np.sum).head(10)
# or state_grouped.aggregate(np.sum)

,OBJECTID,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,HEAT_INDEX,LATITUDE,LONGITUDE
State,,,,,,,,,,,,,,
Alabama,106824,5525.0,2983.9,1402.0,1063.0,8030.0,679,452,0.0,646000.0,41732.7,2637.8,1337.00,-3549.71
Alaska,359143,10979.0,1855.7,1439.9,10202.0,13760.0,2318,1166,847.4,1328500.0,132725.0,0.0,8197.10,-20041.47
Arizona,118604,39289.0,2786.2,557.0,543.0,6960.0,613,542,42.7,663000.0,41688.6,1701.2,1384.27,-4582.96
Arkansas,117193,5993.0,2346.8,1668.6,1796.0,7130.0,498,302,0.0,538000.0,35509.7,303.0,1232.18,-3242.89
California,425766,39208.0,8434.7,3791.4,3914.0,28270.0,2148,1268,519.0,2073600.0,133608.7,2471.2,4722.89,-15663.05
Colorado,155069,132326.0,2370.9,604.1,2115.0,9340.0,985,1005,1956.2,1008000.0,64096.4,0.0,2507.43,-6759.23
Connecticut,38016,632.0,490.9,289.2,457.0,1850.0,188,176,349.9,160000.0,10131.0,0.0,414.92,-727.51
Delaware,12998,60.0,234.8,133.1,154.0,770.0,83,35,0.0,59000.0,4060.5,0.0,156.69,-302.04
Florida,295378,1465.0,7144.1,5204.4,4824.0,15340.0,1355,267,0.0,1496700.0,96875.0,5848.4,2703.97,-7832.29


Sure, above variant is equivalent to `state_grouped.sum()`.
One of the advantages of `agg()` method is the possibility of application of many functions to one and same dataset at once

In [255]:
# at first let's write the function which returns amount of notnull values 
def count_not_null(x):
    return len([i for i in x if pd.notnull(i)])
sdfUSA_grouped[['TEMP', 'R_HUMIDITY', "WIND_SPEED"]].agg([np.sum, np.mean, count_not_null]).head(10)

TEMP             ... WIND_SPEED               
                sum       mean  ...       mean count_not_null
State                           ...                          
Alabama      2572.8  62.751220  ...   8.951220             41
Alaska       1597.3  12.778400  ...  15.867188            128
Arizona      2132.7  50.778571  ...   7.523810             42
Arkansas     2098.6  58.294444  ...  15.194444             36
California   6184.2  47.570769  ...   5.893130            131
Colorado     1656.1  25.876562  ...  13.109375             64
Connecticut   455.0  45.500000  ...  12.700000             10
Delaware      210.3  52.575000  ...  14.000000              4
Florida      6726.3  71.556383  ...   9.463158             95
Georgia      5136.1  65.013924  ...   5.936709             79

[10 rows x 9 columns]

Sure, you may apply many functions to whole DataFrame at once

Another simple aggregation example is to compute the size of each group. This is included in `GroupBy` as the `size()` method. It returns a Series whose indexes are the group names and whose values are the sizes of each group.

In [257]:
sdfUSA_grouped.size()

State
Alabama                41
Alaska                128
Arizona                42
Arkansas               36
California            131
Colorado               64
Connecticut            10
Delaware                4
Florida                95
Georgia                79
Guam                    2
Hawaii                 13
Idaho                  24
Illinois               58
Indiana                41
Iowa                   60
Johnston/Wake/Xmas      2
Kansas                 55
Kentucky               38
Louisiana              51
Maine                  17
Marshall Islands        1
Maryland               22
Massachusetts          23
Michigan               75
Minnesota             100
Mississippi            33
Missouri               43
Montana                38
Nebraska               39
Nevada                 22
New Hampshire          13
New Jersey             17
New Mexico             32
New York               46
North Carolina         69
North Dakota           36
Ohio                   56
Oklaho

>### Exercise 1

> Read in the Gulf_and_Caribbean_Marine_Ecoregions.shp file (loaded into map 1) and also included in the data package.
> Use the groupby functions you learned about to create a dataframe that contains the total area (Shape_Area column) for each of unique values in the Lat_Zone field.


In [258]:
# type your code here

